In [9]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

Purpose of the following project is to:
1. Scrape, clean and format chosen game data from understat.com
2. Allow for a per-team shot visualization based on the scraped data.

In [10]:
#scrape single game
base_url = 'https://understat.com/match/'
match = str(input('Please Enter the Match ID: '))
url = base_url + match

Please Enter the Match ID: 26631


In [11]:
print(url)

https://understat.com/match/26631


In [12]:
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [13]:
scripts

[<script>
 			var THEME = localStorage.getItem("theme") || 'DARK';
 			document.body.className = "theme-" + THEME.toLowerCase();
 		</script>,
 <script>
 	var shotsData 	= JSON.parse('\x7B\x22h\x22\x3A\x5B\x7B\x22id\x22\x3A\x22589903\x22,\x22minute\x22\x3A\x2231\x22,\x22result\x22\x3A\x22MissedShots\x22,\x22X\x22\x3A\x220.8669999694824219\x22,\x22Y\x22\x3A\x220.5170000076293946\x22,\x22xG\x22\x3A\x220.07979319989681244\x22,\x22player\x22\x3A\x22Noussair\x20Mazraoui\x22,\x22h_a\x22\x3A\x22h\x22,\x22player_id\x22\x3A\x2210696\x22,\x22situation\x22\x3A\x22OpenPlay\x22,\x22season\x22\x3A\x222024\x22,\x22shotType\x22\x3A\x22RightFoot\x22,\x22match_id\x22\x3A\x2226631\x22,\x22h_team\x22\x3A\x22Manchester\x20United\x22,\x22a_team\x22\x3A\x22Liverpool\x22,\x22h_goals\x22\x3A\x220\x22,\x22a_goals\x22\x3A\x223\x22,\x22date\x22\x3A\x222024\x2D09\x2D01\x2015\x3A00\x3A00\x22,\x22player_assisted\x22\x3A\x22Alejandro\x20Garnacho\x22,\x22lastAction\x22\x3A\x22Chipped\x22\x7D,\x7B\x22id\x22\x3A\x225899

In [14]:
#get only the shots data
strings = scripts[1].string

In [15]:
strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22589903\\x22,\\x22minute\\x22\\x3A\\x2231\\x22,\\x22result\\x22\\x3A\\x22MissedShots\\x22,\\x22X\\x22\\x3A\\x220.8669999694824219\\x22,\\x22Y\\x22\\x3A\\x220.5170000076293946\\x22,\\x22xG\\x22\\x3A\\x220.07979319989681244\\x22,\\x22player\\x22\\x3A\\x22Noussair\\x20Mazraoui\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x2210696\\x22,\\x22situation\\x22\\x3A\\x22OpenPlay\\x22,\\x22season\\x22\\x3A\\x222024\\x22,\\x22shotType\\x22\\x3A\\x22RightFoot\\x22,\\x22match_id\\x22\\x3A\\x2226631\\x22,\\x22h_team\\x22\\x3A\\x22Manchester\\x20United\\x22,\\x22a_team\\x22\\x3A\\x22Liverpool\\x22,\\x22h_goals\\x22\\x3A\\x220\\x22,\\x22a_goals\\x22\\x3A\\x223\\x22,\\x22date\\x22\\x3A\\x222024\\x2D09\\x2D01\\x2015\\x3A00\\x3A00\\x22,\\x22player_assisted\\x22\\x3A\\x22Alejandro\\x20Garnacho\\x22,\\x22lastAction\\x22\\x3A\\x22Chipped\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22589905\\x22,\\x22minute\\x22\\x3A

In [16]:
#clean data for json
ind_st = strings.index("('")+2
ind_e = strings.index("')")
json_data = strings[ind_st:ind_e]
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert str to json format
data = json.loads(json_data)

In [17]:
data

{'h': [{'id': '589903',
   'minute': '31',
   'result': 'MissedShots',
   'X': '0.8669999694824219',
   'Y': '0.5170000076293946',
   'xG': '0.07979319989681244',
   'player': 'Noussair Mazraoui',
   'h_a': 'h',
   'player_id': '10696',
   'situation': 'OpenPlay',
   'season': '2024',
   'shotType': 'RightFoot',
   'match_id': '26631',
   'h_team': 'Manchester United',
   'a_team': 'Liverpool',
   'h_goals': '0',
   'a_goals': '3',
   'date': '2024-09-01 15:00:00',
   'player_assisted': 'Alejandro Garnacho',
   'lastAction': 'Chipped'},
  {'id': '589905',
   'minute': '38',
   'result': 'SavedShot',
   'X': '0.840999984741211',
   'Y': '0.3340000152587891',
   'xG': '0.03780299797654152',
   'player': 'Noussair Mazraoui',
   'h_a': 'h',
   'player_id': '10696',
   'situation': 'OpenPlay',
   'season': '2024',
   'shotType': 'RightFoot',
   'match_id': '26631',
   'h_team': 'Manchester United',
   'a_team': 'Liverpool',
   'h_goals': '0',
   'a_goals': '3',
   'date': '2024-09-01 15:00:

In [18]:
minute = []
x = []
y = []
xg = []
result = []
team = []
ha = []
data_away = data['a']
data_home = data['h']

In [19]:
for index in range(len(data_home)):
  minute.append(data_home[index]['minute'])
  x.append(data_home[index]['X'])
  y.append(data_home[index]['Y'])
  xg.append(data_home[index]['xG'])
  result.append(data_home[index]['result'])
  ha.append('H')
  team.append(data_home[index]['h_team'])

  minute.append(data_away[index]['minute'])
  x.append(data_away[index]['X'])
  y.append(data_away[index]['Y'])
  xg.append(data_away[index]['xG'])
  result.append(data_away[index]['result'])
  ha.append('A')
  team.append(data_away[index]['a_team'])

In [20]:
#create the dataframe
col_name = ['Minute', 'X','Y', 'xG', 'Result', 'Team', 'H/A']
df = pd.DataFrame([minute,x,y,xg,result,team,ha],index=col_name)
df = df.T

In [21]:
#chronological shot catalog
df["Minute"] = pd.to_numeric(df["Minute"])
df = df.sort_values(by=['Minute'], ascending = True)

# Convert column to numeric type before the operation
df['X'] = pd.to_numeric(df['X'])
df['Y'] = pd.to_numeric(df['Y'])
df['xG'] = pd.to_numeric(df['xG'])
df

,Minute,X,Y,xG,Result,Team,H/A
1,3,0.903,0.658,0.078696,MissedShots,Liverpool,A
3,26,0.872,0.546,0.035418,MissedShots,Liverpool,A
0,31,0.867,0.517,0.079793,MissedShots,Manchester United,H
5,34,0.956,0.565,0.387605,Goal,Liverpool,A
2,38,0.841,0.334,0.037803,SavedShot,Manchester United,H
7,41,0.875,0.462,0.389945,Goal,Liverpool,A
4,51,0.826,0.636,0.017393,SavedShot,Manchester United,H
9,55,0.857,0.420,0.295436,Goal,Liverpool,A
11,56,0.861,0.491,0.173722,MissedShots,Liverpool,A
13,57,0.865,0.293,0.053936,BlockedShot,Liverpool,A


In [22]:
total_shots_home = df[df['H/A'] == 'H'].shape[0]
total_goals_home = df[(df['Result'] == 'Goal') & (df['H/A'] == 'H')].shape[0]
total_shots_away = df[df['H/A'] == 'A'].shape[0]
total_goals_away = df[(df['Result'] == 'Goal') & (df['H/A'] == 'A')].shape[0]
total_xg_home = df[df['H/A'] == 'H']['xG'].sum()
total_xg_away = df[df['H/A'] == 'A']['xG'].sum()

In [23]:
home_team = df[df['H/A'] == 'H']['Team'].iloc[0]
away_team = df[df['H/A'] == 'A']['Team'].iloc[0]
print(f'{home_team} - Shots: {total_shots_home}, Goals: {total_goals_home}, xG: {total_xg_home:.2f}')
print(f'{away_team} - Shots: {total_shots_away}, Goals: {total_goals_away}, xG: {total_xg_away:.2f}')

Manchester United - Shots: 8, Goals: 0, xG: 1.50
Liverpool - Shots: 8, Goals: 3, xG: 1.49
